## Expert Usage for Mixtral 8x7B

After experimenting with the expert hooks in Mixtral 8x7B, I want to see how many tokens of different datasets are sent to which 

In [1]:
import torch as t
import pandas as pd
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
import altair as alt
from mlx_lm import load, generate

import os
from collections import defaultdict
from dotenv import load_dotenv
from huggingface_hub import login

In [4]:
load_dotenv()
login(os.getenv("HF_TOKEN"), add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain,store).
Your token has been saved to /Users/kj3moraes/.cache/huggingface/token
Login successful


## Datasets

I will be using 3 different datasets, and recording the percentages of tokens hanldled by each of the experts in the first and the last layers of the model. Let's see how this differs for different datasets. The datasets are
- `stanfordnlp/imdb` - this is a classification task for movie reviews.
- `databricks/databricks-dolly-15k` - plain 'ol question answering
- `bigcode/bigcodebench` - code generation from prompt

In [6]:
imdb_dataset = load_dataset("stanfordnlp/imdb", split="test")
qa_dataset = load_dataset("databricks/databricks-dolly-15k", split="test")
code_dataset = load_dataset("bigcode/bigcodebench", split="test")

Error while downloading from https://huggingface.co/datasets/stanfordnlp/imdb/resolve/e6281661ce1c48d982bc483cf8a173c1bbeb5d31/plain_text/unsupervised-00000-of-00001.parquet: HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out.
Trying to resume download...


ConnectionError: (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /datasets/stanfordnlp/imdb/resolve/e6281661ce1c48d982bc483cf8a173c1bbeb5d31/plain_text/unsupervised-00000-of-00001.parquet (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x34cdfa890>: Failed to resolve \'huggingface.co\' ([Errno 8] nodename nor servname provided, or not known)"))'), '(Request ID: 8dc96b4c-32ff-4073-8de3-d45014a876bf)')

## Model Setup

In [ ]:
model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=t.float16,
    bnb_4bit_use_double_quant=True
)

model, tokenizer = load(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
NUM_LAYERS = len(model.layers)
NUM_EXPERTS = len(model.layers[0].block_sparse_moe.experts)
print(model)

In [ ]:
first_expert_usage = defaultdict(int) 
last_expert_usage = defaultdict(int)

In [ ]:
def first_expert_update(module, input, output):
    _, topk_index = t.topk(output, 2, dim=1) 
    # topk_list of of the shape [S_l, 2] where S_l is the length of the sequence
    topk_list = topk_index.tolist()

    # iterate over all the tokens in the sequence
    for topk in topk_list: 
        expert_1, expert_2 = tuple(topk) 
        first_expert_usage[expert_1] += 1
        first_expert_usage[expert_2] += 1

def last_expert_update(module, input, output): 
    _, topk_index = t.topk(output, 2, dim=1) 
    # topk_list of of the shape [S_l, 2] where S_l is the length of the sequence
    topk_list = topk_index.tolist()

    # iterate over all the tokens in the sequence
    for topk in topk_list: 
        expert_1, expert_2 = tuple(topk) 
        last_expert_usage[expert_1] += 1
        last_expert_usage[expert_2] += 1


I will only register the hooks for the first and the last experts. We can try and visualize these then.

In [ ]:
model.layers[0].block_sparse_moe.gate.register_forward_hook(first_expert_update)
model.layers[-1].block_sparse_moe.gate.register_forward_hook(last_expert_update)

## Plotting 

Now we can plot these usage charts per dataset. 